In [212]:
! pip install nltk
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet
from nltk import pos_tag, WordNetLemmatizer


amica_df = pd.read_csv('C:/Users/student/Desktop/amica_dataset.csv')
liberty_df = pd.read_csv('C:/Users/student/Desktop/liberty_dataset.csv')


In [213]:
pd.options.mode.chained_assignment = None

I have added the code above as a way of dealing with writing over data frames as we clean our data. Anytime we want to process a data frame with some cleaning algorithm, we may not care about the old data and so will want to simply write-over the old data frame. Although it is not always a best practice to write over old data with new data, it is often more efficient for memory and so I simply suggest using your own discretion. Using the above code will stop Pandas from printing a warning to this effect.

In [214]:
amica_df.head()

,User ID,name,screen_name,location,description,followers_count,friends_count,favourites_count,User Creation Date,gender,Tweet Timestamp,Tweet ID,text,retweet_count,favorite_count,is_quote_status,retweeted
0,989465942500528128,Tim Clarke,timyclarke,"Fort Lauderdale, FL",A daydreamer with unending quest to find what ...,92,662,237,2018-04-26 11:27:18+00:00,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,158562722,Edge Pusher,edgepusher,In the screwed up US of A,"Programmer, writer, mother, musician. Wonders ...",930,2078,2002,2010-06-23 01:23:45+00:00,unknown,Tue Nov 03 02:34:08 +0000 2020,1.323453e+18,@LCarrington259 @rcmahoney @JoeBiden I think i...,0.0,20.0,False,False
2,158562722,Edge Pusher,edgepusher,In the screwed up US of A,"Programmer, writer, mother, musician. Wonders ...",930,2078,2002,2010-06-23 01:23:45+00:00,unknown,Mon Nov 02 20:20:43 +0000 2020,1.323359e+18,@GeorgeTakei Homosexuals overturn our savior D...,0.0,1.0,False,False
3,158562722,Edge Pusher,edgepusher,In the screwed up US of A,"Programmer, writer, mother, musician. Wonders ...",930,2078,2002,2010-06-23 01:23:45+00:00,unknown,Mon Nov 02 18:45:27 +0000 2020,1.323335e+18,@Matthew02388332 @drileyelird @PeteButtigieg A...,0.0,0.0,False,False
4,158562722,Edge Pusher,edgepusher,In the screwed up US of A,"Programmer, writer, mother, musician. Wonders ...",930,2078,2002,2010-06-23 01:23:45+00:00,unknown,Tue Nov 03 02:34:08 +0000 2020,1.323453e+18,@LCarrington259 @rcmahoney @JoeBiden I think i...,0.0,20.0,False,False


In [250]:
text_col = 'text'
group_col = 'gender'
adf_text = amica_df[[group_col, text_col]]

adf_text[text_col] = adf_text[text_col].replace(to_replace=r'[ , | ? | $ | . | ! | - | : ]' , value = r'', regex = True)
adf_text[text_col] = adf_text[text_col].replace(to_replace=r'[ ^a-zA-Z ] ', value = r' ', regex = True)
adf_text[text_col] = adf_text[text_col].replace(to_replace=r'\s\s+' , value = r' ', regex = True)

In [251]:
ldf_text = liberty_df[[group_col, text_col]]

ldf_text[text_col] = ldf_text[text_col].replace(to_replace=r'[ , | ? | $ | . | ! | - | : ]' , value = r'', regex = True)
#ldf_text[text_col] = ldf_text[text_col].replace(to_replace=r'[ ^a-zA-Z ] ', value = r' ', regex = True)
ldf_text[text_col] = ldf_text[text_col].replace(to_replace=r'\s\s+' , value = r' ', regex = True)

In the next set of code we first identify the column that contains our grouper like gender. We then identify our column that contains our text data. Once we have identified our two most important columns we create a new data frame of just those columns called df_text.

Finally, I have included 3 different sets of code for doing some initial processing of the text data using Regex functions. The first function replaces funny symbols with nothing in order to remove funny symbols from analysis. You can add more symbols that may be unique to your data set by adding a | and then the symbol after. The second regex function replaces all non-letters with a space. The last regex pattern removes extra blank spaces and replaces them with a single space to ensure that each word only contains one space to the next word. These obviously have overlapping effects so use one, all, or modify to your specific needs.

In [217]:
adf_text.head(10)

,gender,text
0,male,NaN
1,unknown,@LCarrington259 @rcmahone @JoeBide thin it’ si...
2,unknown,@GeorgeTake Homosexual overtur ou savio Donal ...
3,unknown,@Matthew02388332 @drileyelir @PeteButtigie Als...
4,unknown,@LCarrington259 @rcmahone @JoeBide thin it’ si...
5,unknown,@GeorgeTake Homosexual overtur ou savio Donal ...
6,unknown,@Matthew02388332 @drileyelir @PeteButtigie Als...
7,unknown,NaN
8,mostly_male,NaN
9,unknown,NaN


Error in data type shown below. We must convert the text column to a string type. 

In [218]:
adf_text.text[5]

"@GeorgeTake Homosexual overtur ou savio Donal Trum 😆 He' probabl enjo i "

In [252]:
text= [adf_text, ldf_text]
for df in text:
    df['text'] = df['text'].astype(str)

In [253]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [255]:
wnl = WordNetLemmatizer()
stop = set(nltk.corpus.stopwords.words('english'))

In [254]:
operators = set(['not','n/a','na'])
stopwords = stop - operators

In [225]:
def remove_stopwords(tokens, stopwords):
    return [token for token in tokens if token not in stopwords]

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    if treebank_tag.startswith('V'):
           return wordnet.VERB
    if treebank_tag.startswith('N'):
        return wordnet.NOUN
    if treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

def lemmarati(tup_list):
    if not (np.all(pd.notnull(tup_list))):
        return tup_list
    outputlist = []
    for i, j in tup_list:
        pos = get_wordnet_pos(i)
        lemma = wnl.lemmatize(i)
        outputlist.append(lemma)
    return outputlist

In the next set of code, we are activating and setting up some functions that will allow us to do some more cleaning and normalizing of the text data. More specifically, the code sets up a function to remove stopwords, or words that are very common and as a result not all that meaningful (e.g. the). The remaining code also performs lemmatization. Lemmatization is a way of normalizing text so that words like Python, Pythons, and Pythonic all become just Python. Thus, lemmatization is like stemming but it takes the part of speech into account so that meet (v) and meeting (n) are kept separate.

Also, note that before defining our stopword list we remove some words that we want to keep in our topic analysis. Words like ‘not’ although often considered a stopword, can be very important when performing topic or sentiment analysis. Consider the difference between ‘happy’ and ‘not happy.’ The latter is the opposite of the former however if we used the nltk stopwords list we would remove ‘not’ from the list and run the risk of thinking most comments were ‘happy’ when in reality they were ‘not happy.’

In [226]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [227]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [256]:

adf_text[text_col] = adf_text[text_col].map(lambda x: nltk.word_tokenize(x.lower()) if (np.all(pd.notnull(x))) else x.lower())



In [257]:
adf_text[text_col] = adf_text[text_col].map(lambda x: pos_tag(x) if (np.all(pd.notnull(x))) else x)


In [258]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [259]:
adf_text[text_col] = adf_text[text_col].map(lemmarati)

In [260]:
adf_text[text_col] = adf_text[text_col].map(lambda x: remove_stopwords(x,stopwords) if (np.all(pd.notnull(x))) else x)

In [261]:
adf_text[text_col] = adf_text[text_col].map(lambda x: ' '.join(x) if (np.all(pd.notnull(x))) else x)

In [262]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

n_features = 1000
n_topics= 10

tfidf_vec = TfidfVectorizer(max_df = .95, min_df = 2, max_features = n_features, ngram_range = (2,3))

In [263]:
groups = df_text[group_col].unique()
results = []

for i in groups: 
    df_grp = adf_text.loc[adf_text[group_col] == i]
    if len(df_grp[text_col]) > 3:
        tf = tfidf_vec.fit_transform(df_grp[text_col])
        feature_names = tfidf_vec.get_feature_names()
        try:
            nmf = NMF(n_components = n_topics, random_state=1,alpha=.1, l1_ratio=.5).fit(tf)
            df_topics = pd.DataFrame(nmf.components_)
            df_topics.columns = feature_names
            df_top = df_topics.apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:5].index,index=['top1','top2','top3','top4','top5']), axis=1).reset_index()
            df_top['Group'] = i
            results.append(df_top)
        except:
            results.append(i+' Did not produce topic results')

C:\Users\student\anaconda3new\lib\site-packages\sklearn\decomposition\_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
C:\Users\student\anaconda3new\lib\site-packages\sklearn\decomposition\_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
C:\Users\student\anaconda3new\lib\site-packages\sklearn\decomposition\_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In the code above, we first get a list of the unique groups in our grouping column. We then create a container (in this case a list) to hold our resulting data frames from the NMF topic analysis.

In the for loop, we perform a separate NMF analysis for each unique group contained in the grouping column. We use the ‘if len(df_grp[text_col]) > 100’ logic to ensure we have enough rows of text for the analysis. We use the ‘try:’ statement to ensure that the analysis will still run in case one of the groups gives us an error. In the ‘try:’ code we perform the NMF, extract the components into a data frame, label the data frame with the feature names (the bi and trigrams), selecting only the top 5 bi and trigrams for each topic based on their numeric contribution to the topic, add a column to the data frame to keep track of which group the topics are for, and append the results into our results list.

Now we have a list of data frames, which are not useful as a list so one more step before we finish.

In [264]:
topic_results = pd.concat(results,axis=0)
topic_results.to_csv('C:/Users/student/Desktop/my_aNMF_results.csv')

In [265]:
topic_results.head()

,index,top1,top2,top3,top4,top5,Group
0,0,https tco,https tco rc5z3gfeau,tco rc5z3gfeau,makemoneyonline https,makemoneyonline https tco,male
1,1,realdonaldtrump joebiden,patricksvitek dccc realdonaldtrump,suezq82cubfan patricksvitek,joebiden brandyfortexas senkamalaharris,joebiden brandyfortexas,male
2,2,supermarioodyssey nintendoswitch,zrgsafg58avia klowdtelevision,joebiden brandyfortexas senkamalaharris,italiancharitiesofamericaawarded https tco,italianhorrorcinemathemostinfluentialhorrorfil...,male
3,3,tco dobuyktkpb,zrgsafg58avia klowdtelevision,italiancharitiesofamericaawarded https,italianhorrorcinemathemostinfluentialhorrorfil...,italianhorrorcinemathemostinfluentialhorrorfil...,male
4,4,rt berniesanders,zrgsafg58avia klowdtelevision,italiancharitiesofamericaawarded https tco,italianhorrorcinemathemostinfluentialhorrorfil...,ithoughtyou dbeinterestedinthisstory,male


In [266]:
topic_results.top3[4]

4      italiancharitiesofamericaawarded https tco
4                    whataboutyou haveyouvotedyet
4               carey fratalano matteo_marzolican
4        grainnekelly5 bubblebumukltd theampderry
4    freethoughtkaty quantumentang13 shomaristone
4                                  tco 0rqn7en3av
Name: top3, dtype: object